In [1]:
import pandas as pd
import unicodedata
import requests

# #send to rest API
import json 
import requests
from requests.auth import HTTPBasicAuth

In [2]:
batter_team_2025 = pd.read_csv('../stats/batters_team_2025.csv')
def_rank=pd.read_csv('../../scrapers/def_scraper/batter_hits_def_rank.csv')
under_hit_rate=pd.read_csv('under_hit_rate.csv')
over_hit_rate=pd.read_csv('over_hit_rate.csv')


In [3]:
batter_team_2025['team'].unique()

array(['CHC', 'LAD', 'ARI', 'PIT', 'MIA', 'TOR', 'BOS', 'ATH', 'BAL',
       'Team', 'SEA', 'SDP', 'ATL', 'PHI', 'TEX', 'MIL', 'WSN', 'NYY',
       'STL', 'TBR', 'DET', 'KCR', 'CLE', 'COL', 'NYM', 'CIN', 'MIN',
       'SFG', 'HOU', 'CHW', 'LAA', nan], dtype=object)

In [4]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'ATH': 'ATH',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAAs': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)

In [5]:
def_rank.rename(columns={'Team': 'team'}, inplace=True)


In [6]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'OAK',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank['team'] = def_rank['team'].map(team_mapping)

In [7]:
def_rank

,Unnamed: 0,team,Rank
0,0,NYY,1
1,1,HOU,2
2,2,DET,3
3,3,SDP,4
4,4,CIN,5
5,5,TEX,6
6,6,NYM,7
7,7,KCR,8
8,8,ATL,9
9,9,TBR,10


In [8]:
def_rank.rename(columns={'Rank': 'def_rank'}, inplace=True)


In [9]:
#fixing the accentes on the names
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [10]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [11]:
#cleaning the # and *
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)


In [12]:
# Rows to delete
rows_to_drop = set()

# Loop through the DataFrame
for i, val in batter_team_2025['team'].items():
    if val == '2TM':
        rows_to_drop.update(range(i, i+2))  # this row + 1 more
    elif val == '3TM':
        rows_to_drop.update(range(i, i+3))  # this row + 2 more
    elif val == '4TM':
        rows_to_drop.update(range(i, i+4))  # this row + 3 more
    elif val == '5TM':
        rows_to_drop.update(range(i, i+5))  # this row + 3 more

# Drop rows
batter_team_2025 = batter_team_2025.drop(rows_to_drop).reset_index(drop=True)

In [13]:
# Set API Key
headers = {
    'X-Api-Key': 'fac7e1a69f2baa47da395bde36dc869c'  # Ensure this is correct
}

# API URL
url = 'https://api.sportsgameodds.com/v2/events/'

next_cursor = None
event_data = []

# Fetch only Over/Under odds for Goals + Assists across all players
while True:
    try:
        response = requests.get(
            url,
            headers=headers,
            params={
                "sportID":"BASEBALL",
                "leagueID": "MLB",
                "oddIDs": "batting_hits-PLAYER_ID-game-ou-over",
                "includeOpposingOddIDs": "true", 
                "oddsAvailable":"true",
                "cursor": next_cursor,
                "include":"teams",
                'started':"false",

            },
        )

        response.raise_for_status()  # Raise an error if the request fails

        data = response.json()
        event_data.extend(data.get("data", []))
        next_cursor = data.get("nextCursor")

        if not next_cursor:
            break

    except requests.exceptions.RequestException as e:
        print("Error fetching events:", e)
        break



In [14]:
event_data

[{'eventID': 'n0C2NflrySgFACc5WKoP',
  'sportID': 'BASEBALL',
  'leagueID': 'MLB',
  'type': 'match',
  'odds': {'batting_hits-CALEB_DURBIN_1_MLB-game-ou-over': {'oddID': 'batting_hits-CALEB_DURBIN_1_MLB-game-ou-over',
    'opposingOddID': 'batting_hits-CALEB_DURBIN_1_MLB-game-ou-under',
    'marketName': 'Caleb Durbin Hits Over/Under',
    'statID': 'batting_hits',
    'statEntityID': 'CALEB_DURBIN_1_MLB',
    'periodID': 'game',
    'betTypeID': 'ou',
    'sideID': 'over',
    'playerID': 'CALEB_DURBIN_1_MLB',
    'started': False,
    'ended': False,
    'cancelled': False,
    'bookOddsAvailable': True,
    'fairOddsAvailable': True,
    'fairOdds': '+100',
    'bookOdds': '-180',
    'fairOverUnder': '1',
    'bookOverUnder': '0.5',
    'openFairOdds': '+100',
    'openBookOdds': '-180',
    'openFairOverUnder': '1',
    'openBookOverUnder': '0.5',
    'scoringSupported': True,
    'byBookmaker': {'fanduel': {'odds': '-195',
      'overUnder': '0.5',
      'lastUpdatedAt': '2025-0

In [15]:
# Initialize an empty list for structured data
odds_list = []

for event in event_data:
    event_id = event.get("eventID", "Unknown Event")
    teams = sorted(set(player_data.get("teamID") for player_data in event.get("players", {}).values() if player_data.get("teamID")))
    
    odds = event.get("odds", {})
    
    for odd_id, odd_object in odds.items():
        base_row = {
            "eventID": event_id,
            "teams": ", ".join(teams),
            "oddID": odd_object.get("oddID"),
            "opposingOddID": odd_object.get("opposingOddID"),
            "marketName": odd_object.get("marketName"),
            "playerID": odd_object.get("statEntityID"),
            "side": odd_object.get("sideID"),
            "line": odd_object.get("bookOverUnder", odd_object.get("fairOverUnder", "N/A")),
            "score": odd_object.get("score", "N/A"),
        }
        
        # Extract bookmaker-specific odds into separate rows
        by_bookmaker = odd_object.get("byBookmaker", {})
        
        for bookmaker, book_data in by_bookmaker.items():
            row = base_row.copy()
            row["bookmaker"] = bookmaker
            row["odds"] = book_data.get("odds", "N/A")
            row["overUnder"] = book_data.get("overUnder", "N/A")
            odds_list.append(row)

# Convert to Pandas DataFrame
df_odds = pd.DataFrame(odds_list)

# Print the DataFrame
df_odds.head(5)

,eventID,teams,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder
0,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,fanduel,-195,0.5
1,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,fanatics,-180,0.5
2,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,espnbet,-180,0.5
3,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,draftkings,-180,0.5
4,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,caesars,-194,0.5


In [16]:
# df_odds['overUnder'].value_counts()

In [17]:
# row = df_odds.loc[
#     (df_odds['marketName'] == 'Carlos Correa Hits Over/Under') & 
#     (df_odds['bookmaker'].str.lower() == 'fanduel')
# ]

# if not row.empty:
#     line_value = row['overUnder'].values[0]
#     odds_value = row['odds'].values[0]  # Replace 'odds' with actual column name if different
#     print(f"Line: {line_value}, Odds: {odds_value}")
# else:
#     print("Carlos Correa market at FanDuel not found.")


In [18]:
#getting the last 3 letters of the teams collumn to get MLB
df_odds['sport'] = df_odds['teams'].str[-3:]

In [19]:
# Split into two new columns
df_odds[['team_1', 'team_2']] = df_odds['teams'].str.split(', ', expand=True)
# Remove the "_MLB" suffix from both columns using .loc
df_odds.loc[:, 'team_1'] = df_odds['team_1'].str.replace('_MLB', '', regex=False)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].str.replace('_MLB', '', regex=False)
# Drop the original 'teams' column
df_odds = df_odds.drop(columns=['teams'])

In [20]:
#extract the players name from marketName
df_odds['player'] = df_odds['marketName'].str.extract(r'^(.*?)\s*Hits')

In [21]:
# Define the mapping dictionary
team_mapping = {
    'ARIZONA_DIAMONDBACKS': 'ARI',
    'ATLANTA_BRAVES': 'ATL',
    'BALTIMORE_ORIOLES': 'BAL',
    'BOSTON_RED_SOX': 'BOS',
    'CHICAGO_CUBS': 'CHC',
    'CHICAGO_WHITE_SOX': 'CHW',
    'CINCINNATI_REDS': 'CIN',
    'CLEVELAND_GUARDIANS': 'CLE',
    'COLORADO_ROCKIES': 'COL',
    'DETROIT_TIGERS': 'DET',
    'HOUSTON_ASTROS': 'HOU',
    'KANSAS_CITY_ROYALS': 'KCR',
    'LOS_ANGELES_ANGELS': 'LAA',
    'LOS_ANGELES_DODGERS': 'LAD',
    'MIAMI_MARLINS': 'MIA',
    'MILWAUKEE_BREWERS': 'MIL',
    'MINNESOTA_TWINS': 'MIN',
    'NEW_YORK_METS': 'NYM',
    'NEW_YORK_YANKEES': 'NYY',
    'OAKLAND_ATHLETICS': 'OAK',
    'PHILADELPHIA_PHILLIES': 'PHI',
    'PITTSBURGH_PIRATES': 'PIT',
    'SAN_DIEGO_PADRES': 'SDP',
    'SAN_FRANCISCO_GIANTS': 'SFG',
    'SEATTLE_MARINERS': 'SEA',
    'STLOUIS_CARDINALS': 'STL',
    'TAMPA_BAY_RAYS': 'TBR',
    'TEXAS_RANGERS': 'TEX',
    'TORONTO_BLUE_JAYS': 'TOR',
    'WASHINGTON_NATIONALS': 'WSH'}


# Apply the mapping
df_odds.loc[:, 'team_1'] = df_odds['team_1'].map(team_mapping)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].map(team_mapping)



df_odds.head(4)

,eventID,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder,sport,team_1,team_2,player
0,n0C2NflrySgFACc5WKoP,batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,fanduel,-195,0.5,MLB,CLE,MIL,Caleb Durbin
1,n0C2NflrySgFACc5WKoP,batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,fanatics,-180,0.5,MLB,CLE,MIL,Caleb Durbin
2,n0C2NflrySgFACc5WKoP,batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,espnbet,-180,0.5,MLB,CLE,MIL,Caleb Durbin
3,n0C2NflrySgFACc5WKoP,batting_hits-CALEB_DURBIN_1_MLB-game-ou-over,batting_hits-CALEB_DURBIN_1_MLB-game-ou-under,Caleb Durbin Hits Over/Under,CALEB_DURBIN_1_MLB,over,0.5,N/A,draftkings,-180,0.5,MLB,CLE,MIL,Caleb Durbin


In [22]:
#making the line collumn an integer
df_odds['overUnder'] = pd.to_numeric(df_odds['overUnder'], errors='coerce')

In [23]:
df_odds['bet']='hits'

In [24]:
df_odds=df_odds[['player','side','overUnder','bookmaker','odds','sport','team_1','team_2','bet']]

In [25]:
# Merge on the 'player' column
df_odds = df_odds.merge(
    batter_team_2025[['player', 'team']],
    on='player',
    how='left'
)


In [26]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team
13,Isaac Collins,over,0.5,fanduel,-150,MLB,CLE,MIL,hits,MIL
14,Isaac Collins,over,0.5,espnbet,-150,MLB,CLE,MIL,hits,MIL
15,Isaac Collins,over,0.5,draftkings,-150,MLB,CLE,MIL,hits,MIL
16,Isaac Collins,over,0.5,caesars,-154,MLB,CLE,MIL,hits,MIL
17,Isaac Collins,over,0.5,hardrockbet,-150,MLB,CLE,MIL,hits,MIL
...,...,...,...,...,...,...,...,...,...,...
4327,Pavin Smith,under,0.5,betmgm,+100,MLB,ARI,SFG,hits,ARI
4328,Pavin Smith,under,0.5,prophetexchange,+107,MLB,ARI,SFG,hits,ARI
4329,Geraldo Perdomo,under,0.5,fanatics,+115,MLB,ARI,SFG,hits,ARI
4330,Geraldo Perdomo,under,0.5,betonline,+118,MLB,ARI,SFG,hits,ARI


In [27]:
# Step 1: Identify the "odd one out" team to find the defensive team
df_odds['def_team'] = df_odds.apply(lambda row: row['team_2'] if row['team'] == row['team_1'] else row['team_1'], axis=1)

In [28]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team,def_team
13,Isaac Collins,over,0.5,fanduel,-150,MLB,CLE,MIL,hits,MIL,CLE
14,Isaac Collins,over,0.5,espnbet,-150,MLB,CLE,MIL,hits,MIL,CLE
15,Isaac Collins,over,0.5,draftkings,-150,MLB,CLE,MIL,hits,MIL,CLE
16,Isaac Collins,over,0.5,caesars,-154,MLB,CLE,MIL,hits,MIL,CLE
17,Isaac Collins,over,0.5,hardrockbet,-150,MLB,CLE,MIL,hits,MIL,CLE
...,...,...,...,...,...,...,...,...,...,...,...
4327,Pavin Smith,under,0.5,betmgm,+100,MLB,ARI,SFG,hits,ARI,SFG
4328,Pavin Smith,under,0.5,prophetexchange,+107,MLB,ARI,SFG,hits,ARI,SFG
4329,Geraldo Perdomo,under,0.5,fanatics,+115,MLB,ARI,SFG,hits,ARI,SFG
4330,Geraldo Perdomo,under,0.5,betonline,+118,MLB,ARI,SFG,hits,ARI,SFG


In [29]:
#drop redundant collumns that are useless
df_odds = df_odds.drop(columns=['team_1', 'team_2'], errors='ignore')

In [30]:
# Rename the team column before merging (if needed)
def_rank = def_rank.rename(columns={"team": "def_team"})

In [31]:
# Merge on the 'player' column
df_odds = df_odds.merge(def_rank, on='def_team', how='left')
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,bet,team,def_team,Unnamed: 0,def_rank
13,Isaac Collins,over,0.5,fanduel,-150,MLB,hits,MIL,CLE,24,24
14,Isaac Collins,over,0.5,espnbet,-150,MLB,hits,MIL,CLE,24,24
15,Isaac Collins,over,0.5,draftkings,-150,MLB,hits,MIL,CLE,24,24
16,Isaac Collins,over,0.5,caesars,-154,MLB,hits,MIL,CLE,24,24
17,Isaac Collins,over,0.5,hardrockbet,-150,MLB,hits,MIL,CLE,24,24
...,...,...,...,...,...,...,...,...,...,...,...
4327,Pavin Smith,under,0.5,betmgm,+100,MLB,hits,ARI,SFG,10,11
4328,Pavin Smith,under,0.5,prophetexchange,+107,MLB,hits,ARI,SFG,10,11
4329,Geraldo Perdomo,under,0.5,fanatics,+115,MLB,hits,ARI,SFG,10,11
4330,Geraldo Perdomo,under,0.5,betonline,+118,MLB,hits,ARI,SFG,10,11


In [32]:
#making game collumn
df_odds['game'] = df_odds['team'] + ' vs ' + df_odds['def_team']


In [33]:
df_odds2=df_odds.dropna()

In [34]:
df_odds3 = df_odds2.drop(columns=['team', 'def_team'])
df_odds3

,player,side,overUnder,bookmaker,odds,sport,bet,Unnamed: 0,def_rank,game
13,Isaac Collins,over,0.5,fanduel,-150,MLB,hits,24,24,MIL vs CLE
14,Isaac Collins,over,0.5,espnbet,-150,MLB,hits,24,24,MIL vs CLE
15,Isaac Collins,over,0.5,draftkings,-150,MLB,hits,24,24,MIL vs CLE
16,Isaac Collins,over,0.5,caesars,-154,MLB,hits,24,24,MIL vs CLE
17,Isaac Collins,over,0.5,hardrockbet,-150,MLB,hits,24,24,MIL vs CLE
...,...,...,...,...,...,...,...,...,...,...
4327,Pavin Smith,under,0.5,betmgm,+100,MLB,hits,10,11,ARI vs SFG
4328,Pavin Smith,under,0.5,prophetexchange,+107,MLB,hits,10,11,ARI vs SFG
4329,Geraldo Perdomo,under,0.5,fanatics,+115,MLB,hits,10,11,ARI vs SFG
4330,Geraldo Perdomo,under,0.5,betonline,+118,MLB,hits,10,11,ARI vs SFG


In [35]:
#splitting overs and unders
overs = df_odds3[(df_odds3['side'].str.lower() == 'over') & (df_odds3['def_rank'] > 15)]
unders = df_odds3[(df_odds3['side'].str.lower() == 'under') & (df_odds3['def_rank'] < 16)]
# overs

In [36]:
overs = overs.merge(over_hit_rate, on='player', how='left')
unders = unders.merge(under_hit_rate, on='player', how='left')
# overs

In [37]:
overs_2=overs.dropna()
unders_2=unders.dropna()

In [38]:
#making odds collumn numeric
unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')

C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\2110421890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\2110421890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')


In [39]:
#making decimal odds
unders_2['decimal_odds'] = unders_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
overs_2['decimal_odds'] = overs_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
# overs_2

C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\3651279232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['decimal_odds'] = unders_2['odds'].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\3651279232.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['decimal_odds'] = overs_2['odds'].apply(


In [40]:
#making profit
overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100

C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\3093691362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\3093691362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100


In [41]:
#edge for overs

def calculate_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_hit_rate_over']
    elif row['overUnder'] == 1.5:
        rate = row['2_hit_rate_over']
    else:
        return None  # or np.nan, if you want to leave it blank for other lines

    # Expected Value formula
    return (rate * row['profit']) - ((1 - rate) * 100)

# Apply the formula to each row
overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\1032727930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


In [42]:
#edge for unders
def calculate_under_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_hit_rate_under']
    elif row['overUnder'] == 1.5:
        rate = row['2_hit_rate_under']
    else:
        return None  # or np.nan for unsupported lines

    return ((1 - rate) * row['profit']) - (rate * 100)

# Apply to each row
unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)



C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\2907333862.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)


In [43]:
overs_2 = overs_2[overs_2['edge'] > 20]

In [44]:
unders_2 = unders_2[unders_2['edge'] > 20]

In [45]:
#merging overs_2 on top of unders_2
final_df = pd.concat([overs_2, unders_2], ignore_index=True)


In [46]:
final_df=final_df[['player','side','overUnder','bookmaker','odds','sport','bet','edge','game']]

In [47]:
final_df['bet'] = final_df['side'].astype(str) + ' ' + final_df['overUnder'].astype(str) + ' ' + final_df['bet'].astype(str)


In [48]:
#need to add + in front of positive odds 

In [49]:
#rename bookmaker
final_df = final_df.rename(columns={'bookmaker': 'book'})

In [50]:
#divide the edge in half
final_df['edge'] = final_df['edge'] / 2

In [51]:
#round the edge to 1 decimal
final_df['edge'] = final_df['edge'].round(1)

In [52]:
final_df=final_df[['book','sport','player','game','bet','odds','edge']]

In [53]:
final_df = final_df.sort_values(by='edge', ascending=False).reset_index(drop=True)


In [54]:
#putting + in front of the ones that dont have it
final_df['odds'] = final_df['odds'].apply(lambda x: f"+{x}" if x > 0 else str(x))

final_df

,book,sport,player,game,bet,odds,edge
0,bovada,MLB,Julio Rodriguez,SEA vs NYY,under 0.5 hits,+185,32.5
1,prophetexchange,MLB,Julio Rodriguez,SEA vs NYY,under 0.5 hits,+182,31.6
2,hardrockbet,MLB,Julio Rodriguez,SEA vs NYY,under 0.5 hits,+180,31.1
3,betmgm,MLB,Julio Rodriguez,SEA vs NYY,under 0.5 hits,+180,31.1
4,caesars,MLB,Julio Rodriguez,SEA vs NYY,under 0.5 hits,+178,30.5
...,...,...,...,...,...,...,...
149,espnbet,MLB,Kyle Farmer,COL vs TEX,under 0.5 hits,+115,10.2
150,draftkings,MLB,Kyle Farmer,COL vs TEX,under 0.5 hits,+115,10.2
151,betmgm,MLB,Mike Yastrzemski,SFG vs ARI,over 0.5 hits,-175,10.1
152,caesars,MLB,Francisco Lindor,NYM vs PIT,over 0.5 hits,-224,10.0


In [55]:
import json
import requests

# Convert your DataFrame to JSON
data_json = json.loads(final_df.to_json(orient="records"))

# Endpoint URL
url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_hits"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request without authentication
response = requests.post(url, headers=headers, json=data_json)

# Debugging output
print("Status Code:", response.status_code)
print("Response Text:", response.text)


Status Code: 200
Response Text: {"message":"Data received","data":[{"book":"bovada","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+185","edge":32.5},{"book":"prophetexchange","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+182","edge":31.6},{"book":"hardrockbet","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+180","edge":31.1},{"book":"betmgm","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+180","edge":31.1},{"book":"caesars","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+178","edge":30.5},{"book":"espnbet","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+175","edge":29.6},{"book":"draftkings","sport":"MLB","player":"Julio Rodriguez","game":"SEA vs NYY","bet":"under 0.5 hits","odds":"+175","edge":29.6},{"book":"fliff","

In [56]:
# # Convert DataFrame to JSON (ensure it's a Python list, not a string)
# data_json = json.loads(final_df.to_json(orient="records"))  # ✅ Converts string to Python object

# # WordPress API URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_hits"

# # Authentication
# auth = HTTPBasicAuth("josheskragmailcom", "UhiM WLwm dEOK T171 RUkT Sw2F")
# # auth = HTTPBasicAuth("josheskragmailcom", "BFgx z2iv nW9I voXn 1X51 fjRq")
# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # Send POST request (requests will handle JSON encoding automatically)
# response = requests.post(url, headers=headers, auth=auth, json=data_json)  # ✅ FIX: Use json= instead of data=

# # Debugging Output
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [57]:
# Step 1: Sort to ensure reproducibility in tie cases
df_sorted = final_df.sort_values('edge')

# Step 2: Define a function to get the row with median edge for each group
def get_median_row(group):
    median_edge = group['edge'].median()
    # Get the row closest to the median (or first in case of exact match/ties)
    return group.iloc[(group['edge'] - median_edge).abs().argsort().iloc[0]]

# Step 3: Apply it to each group
df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)

# Step 4: Reset index if needed
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

C:\Users\Joshua\AppData\Local\Temp\ipykernel_78596\1266780210.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)


,book,sport,player,game,bet,odds,edge
0,prophetexchange,MLB,Alex Bregman,BOS vs DET,under 0.5 hits,+198,22.4
1,betonline,MLB,Connor Wong,BOS vs DET,under 0.5 hits,+145,14.7
2,caesars,MLB,Dylan Crews,WSH vs ATL,under 0.5 hits,+133,11.7
3,ballybet,MLB,Francisco Lindor,NYM vs PIT,over 0.5 hits,-215,10.8
4,fanatics,MLB,Gleyber Torres,DET vs BOS,over 0.5 hits,-255,10.3
5,espnbet,MLB,Isaac Paredes,HOU vs KCR,under 0.5 hits,+130,12.7
6,betonline,MLB,Jacob Young,WSH vs ATL,under 0.5 hits,+130,21.9
7,betonline,MLB,Jake Meyers,HOU vs KCR,under 0.5 hits,+142,25.6
8,hardrockbet,MLB,Jared Triolo,PIT vs NYM,under 0.5 hits,+105,10.7
9,fliff,MLB,Jonathan India,KCR vs HOU,under 0.5 hits,+140,16.0


In [58]:
# Make sure odds column is numeric
df_filtered['odds'] = pd.to_numeric(df_filtered['odds'], errors='coerce')
# Convert American odds to decimal odds
def american_to_decimal(odds):
    if pd.isna(odds):
        return None
    if odds > 0:
        return round((odds / 100) + 1, 3)
    else:
        return round((100 / abs(odds)) + 1, 3)

# Apply conversion
df_filtered['decimal_odds'] = df_filtered['odds'].apply(american_to_decimal)

In [59]:
df_filtered['profit'] = (df_filtered['decimal_odds'] * 100) - 100
df_filtered

,book,sport,player,game,bet,odds,edge,decimal_odds,profit
0,prophetexchange,MLB,Alex Bregman,BOS vs DET,under 0.5 hits,198,22.4,2.980,198.0
1,betonline,MLB,Connor Wong,BOS vs DET,under 0.5 hits,145,14.7,2.450,145.0
2,caesars,MLB,Dylan Crews,WSH vs ATL,under 0.5 hits,133,11.7,2.330,133.0
3,ballybet,MLB,Francisco Lindor,NYM vs PIT,over 0.5 hits,-215,10.8,1.465,46.5
4,fanatics,MLB,Gleyber Torres,DET vs BOS,over 0.5 hits,-255,10.3,1.392,39.2
5,espnbet,MLB,Isaac Paredes,HOU vs KCR,under 0.5 hits,130,12.7,2.300,130.0
6,betonline,MLB,Jacob Young,WSH vs ATL,under 0.5 hits,130,21.9,2.300,130.0
7,betonline,MLB,Jake Meyers,HOU vs KCR,under 0.5 hits,142,25.6,2.420,142.0
8,hardrockbet,MLB,Jared Triolo,PIT vs NYM,under 0.5 hits,105,10.7,2.050,105.0
9,fliff,MLB,Jonathan India,KCR vs HOU,under 0.5 hits,140,16.0,2.400,140.0
